# **Exploratory and Time Series Analysis in SQL**

![Intro Image](.\Materials\IntroImage2.jpg)

An important part

**USAGE STATS**

> **<u>column</u>                                         <u>type</u>                  <u>meaning</u>**
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Date&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;The date of the recorded observation in standart SQL format (YYYY-MM-DD).</span>
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Rented_Bike_Count&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;The count of bikes rented within a set hour.</span>
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Hour&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;The start day-hour of counting from 0 to 23.</span>
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Temperature_C&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;The temperature in Celcius Degrees based 0° the point of freezing water and 100° of boiling water.&nbsp;&nbsp;</span> 
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Humidity&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;Percentage of amount of water vapor in the air.</span>
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Wind_speed_m_s&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;The strength of natural wind affecting the environment measured in meters per second (m/s).</span>
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Visibility_10m&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;Description of the business category</span>
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Rainfall_mm&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;Description of the business category</span>
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Snowfall_cm&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;Description of the business category</span>
> 
> <span style="background-color: rgba(127, 127, 127, 0.1);">Holiday&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; varchar&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;Description of the business category</span>

In [141]:
-- DATASET OVERVIEW 
SELECT TOP 2 *
FROM UsageStats u

(2 rows affected)

Total execution time: 00:00:00.030

Date,Rented_Bike_Count,Hour,Temperature_C,Humidity,Wind_speed_m_s,Visibility_10m,Dew_point_temperature_C,Solar_Radiation_MJ_m2,Rainfall_mm,Snowfall_cm,Seasons,Holiday,Functioning_Day
2017-12-01,254,0,-5.199999809265137,37,2.200000047683716,2000,-17.600000381469727,0,0,0,Winter,No Holiday,Yes
2017-12-01,204,1,-5.5,38,0.800000011920929,2000,-17.600000381469727,0,0,0,Winter,No Holiday,Yes


### **SUMMARY STATISTICS**

As I believe there is no one-only built-in T-SQL function that can provide a summary for all (or selected) dimensions, I had to improvise in my querying to generate key statistics (and a few customisable info) for everything all at once.

In [13]:
                           /* Used UNION clause to arrange the statistics in a Matrix style. */
SELECT '5) Average' 'Statistics',
    AVG(Rented_Bike_Count) Rented_Bike_Count, AVG(Temperature_C) Temperature_C, AVG(Humidity) Humidity, AVG(Wind_speed_m_s) Wind_speed_m_s,
    AVG(Visibility_10m) Visibility_10m, AVG(Rainfall_mm) Rainfall_mm, AVG(Snowfall_cm) Snowfall_cm
FROM UsageStats
UNION                      /* The CASE statement works around the lack of a built-in function to count null values. */
SELECT '1) Null Values',
    SUM(case when Rented_Bike_Count is null then 1 else 0 end), SUM(case when Temperature_C is null then 1 else 0 end),
    SUM(case when Humidity is null then 1 else 0 end), SUM(case when Wind_speed_m_s is null then 1 else 0 end),
    SUM(case when Visibility_10m is null then 1 else 0 end), SUM(case when Rainfall_mm is null then 1 else 0 end),
    SUM(case when Snowfall_cm is null then 1 else 0 end)
FROM UsageStats
UNION                      /* Aggregates functions of AVG, MIN and MAX are also added in the summary. */
SELECT '2) Minimum',
    MIN(Rented_Bike_Count), MIN(Temperature_C), MIN(Humidity), MIN(Wind_speed_m_s), MIN(Visibility_10m), MIN(Rainfall_mm), MIN(Snowfall_cm)
FROM UsageStats
UNION                      /* Had to use WITHIN GROUP and OVER clause as required by the MS documentation to work around getting the median */
SELECT '3) Median',
    PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY Rented_Bike_Count)OVER(), PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY Temperature_C)OVER(),
    PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY Humidity)OVER(), PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY Wind_speed_m_s)OVER(),
    PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY Visibility_10m)OVER(), PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY Rainfall_mm)OVER(),
    PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY Snowfall_cm)OVER()
FROM UsageStats
UNION
SELECT '4) Maximum',
    MAX(Rented_Bike_Count), MAX(Temperature_C), MAX(Humidity), MAX(Wind_speed_m_s), MAX(Visibility_10m), MAX(Rainfall_mm), MAX(Snowfall_cm)
FROM UsageStats
UNION                      /* Standard Deviation also helps me interpret the distribution of the data points. */
SELECT '6) Standard Deviation',
    STDEV(Rented_Bike_Count), STDEV(Temperature_C), STDEV(Humidity), STDEV(Wind_speed_m_s), STDEV(Visibility_10m), STDEV(Rainfall_mm),
    STDEV(Snowfall_cm)
FROM UsageStats

ORDER BY [Statistics]

(6 rows affected)

Total execution time: 00:00:00.152

Statistics,Rented_Bike_Count,Temperature_C,Humidity,Wind_speed_m_s,Visibility_10m,Rainfall_mm,Snowfall_cm
1) Null Values,0,0,0,0,0,0,0
2) Minimum,3,-17.799999237060547,15,0,59,0,0
3) Median,210,-2.5999999046325684,49,1.600000023841858,1700,0,0
4) Maximum,937,8.600000381469727,96,6.699999809265137,2000,9.5,5.099999904632568
5) Average,228,-2.864845943217184,51,1.8015406177919202,1388,0.028291316640352,0.2673669470740216
6) Standard Deviation,146.62096491999318,5.546107643385477,19.10581216026551,1.127423531149696,660.8855099962893,0.35208561890391105,0.6664864109786526


In [94]:
-- DISTRIBUTION OF RENTAL ACTIVITY ACROSS A WEEK
SELECT DayOfWeek, AVG(n) AS 'Average Per Day of Week'
FROM
    (SELECT
        DATEPART(WEEKDAY,date) AS DayNumber,
        DATENAME(WEEKDAY,date) AS DayOfWeek,
        SUM(Rented_Bike_Count) AS n
     FROM UsageStats GROUP BY date) temp
GROUP BY DayOfWeek, DayNumber ORDER BY DayNumber

(7 rows affected)

Total execution time: 00:00:00.027

DayOfWeek,Average Per Day of Week
Sunday,4069
Monday,4843
Tuesday,5796
Wednesday,5340
Thursday,5905
Friday,6788
Saturday,5332


**COMMENT ON RESULTS:** Surprisingly, the numbers for Saturdays and Sundays are below those recorded for most business days, with Sundays performing 25% below the overall average (\<5427) and Fridays proving to be the busiest days with bike sharing trips performing 25% above the same average.

In [69]:
-- DISTRIBUTION OF RENTAL ACTIVITY ACROSS THE DAY
SELECT 
    Hour,
    AVG(Rented_Bike_Count) AS 'Average Rent Count'
FROM UsageStats
GROUP BY Hour
ORDER BY Hour

(24 rows affected)

Total execution time: 00:00:00.053

Hour,Average Rent Count
0,176
1,169
2,127
3,83
4,53
5,54
6,92
7,207
8,421
9,257


**COMMENT ON RESULTS:** Consistent usage above average counts (\>228) only happens between 11am and 10pm and the count decreases significantly after 1h (with the background between 3am and 6am). However, large spikes in the number of rented bikes can be observed at both sunrise (≈7:40am) and sunset (≈5:15pm) [times](https://www.timeanddate.com/sun/south-korea/seoul?month=12&year=2017) in Seoul.

In [140]:
--CATEGORISING WIND SPEED
WITH cte AS ( 
SELECT *, CASE
        WHEN Temperature_C BETWEEN 0 AND 8.9 THEN 'Acceptable'
        WHEN Temperature_C BETWEEN -8.900 AND -0.001 THEN 'Difficult'
        WHEN Temperature_C BETWEEN -17.800 AND -8.790 THEN 'Impossible'
        ELSE 'Unsafe to Travel'
    END AS SeverityOfTemperature
FROM UsageStats)

/* After using a common table expression to bin temperature data into categories,
   I can then retrieve the data into a more visibly, clear way. */

SELECT SeverityOfTemperature, AVG(Rented_Bike_Count) AvgRentedBikes
FROM cte GROUP BY SeverityOfTemperature ORDER BY AvgRentedBikes DESC

(3 rows affected)

Total execution time: 00:00:00.025

SeverityOfTemperature,AvgRentedBikes
Acceptable,275
Difficult,218
Impossible,147


  

Numbers are based on an estimation inspired by the [US National Weather Service](https://www.weather.gov/pqr/wind).

In [74]:
--CATEGORISING WIND SPEED
WITH cte AS ( 
SELECT *, CASE
        WHEN Wind_speed_m_s BETWEEN 0 AND 1.699 THEN 'Calmness to Light Air'
        WHEN Wind_speed_m_s BETWEEN 1.700 AND 4.149 THEN 'Light to Gentle Breeze'
        WHEN Wind_speed_m_s BETWEEN 4.150 AND 7.000 THEN 'Moderate to Strong Breeze'
        ELSE 'Unsafer to Travel'
    END AS WindStrength
FROM UsageStats)

/* After using a common table expression to bin wind speed data into categories,
   I can then retrieve the data into a more visibly, clear way. */

SELECT WindStrength, AVG(Rented_Bike_Count) AvgRentedBikes
FROM cte GROUP BY WindStrength ORDER BY AvgRentedBikes DESC

(3 rows affected)

Total execution time: 00:00:00.029

WindStrength,AvgRentedBikes
Moderate to Strong Breeze,238
Calmness to Light Air,231
Light to Gentle Breeze,222


**COMMENT ON RESULTS:** It is important to note that the dataset doesn't contain any recorded events with wind speed over 7 m/s. Which could be explained by Seoul having its windiest month